In [1]:
import os
import numpy as np
import cv2
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import math
from functools import reduce
from scipy.stats import bartlett
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm

In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mpl

In [2]:
all_path = '/home/jupyter-fly_cv/Igor/Plants/Dataset_for_ResNet_NEW/All/'
first_path = '/home/jupyter-fly_cv/Igor/Plants/Dataset_for_ResNet_NEW/first/'
second_path = '/home/jupyter-fly_cv/Igor/Plants/Dataset_for_ResNet_NEW/second/'

img_path = '/home/jupyter-fly_cv/Igor/Plants/data/Новый признак'
img_path_with_corr = ''

global_path = '/home/jupyter-fly_cv/Igor/Plants/Dataset_for_ResNet_NEW/'

In [3]:
train = ['IMG_8409','IMG_8408','IMG_8407','IMG_8406','IMG_8405','IMG_8404',
        'IMG_8403','IMG_8402','IMG_8401','IMG_8400','IMG_8399','IMG_8398',
        'IMG_8397','IMG_8396','IMG_8395','IMG_8394','IMG_8393','IMG_8392',
        'IMG_8391','IMG_8390',
        'IMG_8440','IMG_8439','IMG_8438','IMG_8437','IMG_8436','IMG_8435',
        'IMG_8434','IMG_8433','IMG_8432','IMG_8431','IMG_8430','IMG_8410',
        'IMG_8344','IMG_8345','IMG_8346','IMG_8347','IMG_8348','IMG_8423',
        'IMG_8422','IMG_8421',
         
        'IMG_8428', 'IMG_8427','IMG_8426','IMG_8425','IMG_8424',
        'IMG_8339','IMG_8340','IMG_8341','IMG_8342','IMG_8349',
        'IMG_8329','IMG_8330',
        'IMG_8331','IMG_8332','IMG_8333','IMG_8334','IMG_8335','IMG_8336',
        'IMG_8337','IMG_8338',
        'IMG_8379','IMG_8378','IMG_8377','IMG_8376','IMG_8375','IMG_8374',
        'IMG_8373','IMG_8372','IMG_8371','IMG_8370','IMG_8369','IMG_8368',
        'IMG_8367', 'IMG_8366','IMG_8365','IMG_8364','IMG_8363','IMG_8362',
        'IMG_8361','IMG_8350']

val =['IMG_8389','IMG_8388','IMG_8387','IMG_8386','IMG_8385',
     'IMG_8420','IMG_8419','IMG_8418','IMG_8417','IMG_8416',#
     
      'IMG_8359','IMG_8358','IMG_8357','IMG_8356','IMG_8360', #
    'IMG_8324','IMG_8325','IMG_8326','IMG_8327','IMG_8328',]

test=['IMG_8384','IMG_8383','IMG_8382','IMG_8381','IMG_8380',
     'IMG_8415','IMG_8414','IMG_8413','IMG_8412','IMG_8411',
     
     'IMG_8319','IMG_8320','IMG_8321','IMG_8322','IMG_8323',
     'IMG_8354','IMG_8353','IMG_8352','IMG_8351','IMG_8355']

In [4]:
def direct_name(path):
    names = path.split('/')
    direct_path = ''
    for p in names:
        if p == names[-1]:
            break
        direct_path += p + '/'
    return direct_path

In [5]:
def sample_name(path):
    names = path.split('/')
    names = names[-1].split('.')
    return names[0] 

In [6]:
def filename_pref_png(name, pref):
    names = name.split('/')
    png_names = names[-1].split('.')
    names[-1] = png_names[0] + '.' + 'png'
    names[-1] = pref + names[-1]
    new_name = ''
    for n in names:
        if n != names[-1]:
            new_name =new_name + n + '/'
        else:
            new_name =new_name + n
    return new_name    

In [7]:
def mask_name(path):
    dir_name = direct_name(path)
    s_name = sample_name(path)
    return dir_name + filename_pref_png(s_name, 'mask_')

In [8]:
name = '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-2.jpg'
#get_ploidn(f_annot, sample_name(name))
sample_name(name)

'452-3-2'

In [9]:
def all_jpg_img(path):
    img_names = []

    for dirname, _, filenames in os.walk(path):
        for filename in filenames: 
            name = os.path.join(filename)
            if name.split('.')[-1] != 'JPG':
                continue
            img_names.append(dirname + '/' + name)
            
    return img_names

In [10]:
def img_seg(mask,img):
    seg_img = img.copy()
    new_mask = np.clip(mask,0,1)
    for i in range(3):
        seg_img[:,:,i] *=new_mask
    return seg_img

In [11]:
def first_time_point(path):
    names = path.split('/')
    for name in names:
        if name == '2022_03_11 Wheat':
            return True
    return False

In [12]:
def second_time_point(path):
    names = path.split('/')
    for name in names:
        if name == '2022_03_24 Wheat':
            return True
    return False

In [13]:
def genotype(path):
    names = path.split('/')
    return names[-2]

In [14]:
def read_mask(path,global_path):
    if global_path == img_path_with_corr:
        names = path.split('/')
        new_name = ''
        for name in names:
            if name == 'color_correction_01.02.22':
                name = 'Гаплоиды'
            if name != names[-1]:
                new_name += name + '/'
            else:
                new_name += name
        return mask_name(new_name)
    
    if global_path == img_path:
        return mask_name(path)

In [19]:
def rectangle(mask):
    new_mask = np.where(mask > 0, 1,0)
    non_zero_mask = np.nonzero(new_mask)
    h = non_zero_mask[0].max() - non_zero_mask[0].min()
    w = non_zero_mask[1].max() - non_zero_mask[1].min()
    x = set(non_zero_mask[1])
    y = set(non_zero_mask[0])
    x = sum(x)/len(x)
    y = sum(y)/len(y)
    return w,h,x,y

In [33]:
def rescale(img,mask):
    new_img = np.zeros((2075,2075,3),dtype = np.uint8)
    
    w,h,x,y = rectangle(mask)
    w_2 = int(w/2)
    h_2 = int(h/2)
    x = int(x)
    y = int(y)
    
    print(w,h)
    
    s_img = img[y - h_2:y + h_2,x - w_2:x + w_2,:]
    new_img[0: s_img.shape[0], 0: s_img.shape[1]] = s_img
    
    return new_img

In [16]:
img_names = all_jpg_img(global_path)

for name in img_names:
    print(name)
    img = cv2.imread(name) 
    img = rescale(img)
    os.chdir(direct_name(name))
    cv2.imwrite(sample_name(name)+ '.jpg', img)

In [37]:
def prep_data(img_path_from, img_path_to):
    
    train_l_25 = 0
    train_1102 = 0
    val_l_25 = 0
    val_1102 = 0
    test_l_25 = 0
    test_1102 = 0
    
    img_names = all_jpg_img(img_path_from)

    for name in img_names:
        #name = '/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_05/451-06.jpg'
        
        if(first_time_point(name)):
            continue
        
        print(name)
            
        img = cv2.imread(name)
        mask = cv2.imread(read_mask(name,img_path_from),cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print('mask error')
            continue
        img_res = cv2.resize(img,(int(mask.shape[1]), int(mask.shape[0])), interpolation = cv2.INTER_AREA)
        print(img_res.shape)
       # plt.imshow(mask,cmap = 'gray')
        seg_img = rescale(img_seg(mask,img_res),mask)
    

        if set([sample_name(name)]).issubset(set(train)):
            gen = genotype(name)
            if gen == 'L-25':
                train_l_25 += 1
            if gen == '1102':
                train_1102 += 1
            os.chdir(img_path_to + 'train/' + gen )
            cv2.imwrite(sample_name(name)+ '.jpg', seg_img)
        
        if set([sample_name(name)]).issubset(set(val)):
            gen = genotype(name)
            if gen == 'L-25':
                val_l_25 += 1
            if gen == '1102':
                val_1102 += 1
            os.chdir(img_path_to + 'val/' + gen)
            cv2.imwrite(sample_name(name) + '.jpg', seg_img)
        
        if set([sample_name(name)]).issubset(set(test)):
            gen = genotype(name)
            if gen == 'L-25':
                test_l_25 += 1
            if gen == '1102':
                test_1102 += 1
            os.chdir(img_path_to + 'test/' + gen)
            cv2.imwrite(sample_name(name)+ '.jpg', seg_img)
            
    return  train_l_25, train_1102, val_l_25 ,val_1102 ,test_l_25 ,test_1102   
        

In [38]:
train_l_25, train_1102, val_l_25 ,val_1102 ,test_l_25 ,test_1102 = prep_data(img_path, second_path)

/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8392.JPG
(1465, 1453, 3)
991 1278
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8382.JPG
(1542, 1512, 3)
1008 1457
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8394.JPG
(1765, 1915, 3)
1672 1720
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8385.JPG
(1684, 1745, 3)
1070 1639
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8406.JPG
(1467, 1411, 3)
827 1385
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8388.JPG
(1790, 1564, 3)
1123 1670
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8409.JPG
(2028, 1612, 3)
1351 1814
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8403.JPG
(1983, 1536, 3)
846 1762
/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/IMG_8404.JPG
(1465, 1449,

In [ ]:
train_l_25

In [ ]:
train_1102

In [ ]:
val_l_25

In [ ]:
val_1102

In [ ]:
test_l_25

In [ ]:
test_1102